In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlite3
import hashlib
from sqlalchemy import text, inspect, MetaData, Table
from sqlalchemy.orm import sessionmaker
from app.main import getTables, Base, Account, Transaction, TransactionFinal, Load
from app.helpers.readCsv import openOneCsv, getAccountHistory
from app.helpers.sqlEngine import engine

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)

Session = sessionmaker(bind=engine)
session = Session()

In [9]:
accounts = getTables(engine)
session.rollback()
for index, account in accounts.iterrows():
    path = account['csvPath']
    print(path)
    query = f"SELECT * FROM Loads WHERE accountId = {account['id']};"
    print(query, account['id'])
    allFiles = getAccountHistory(path, account['id'], session)
    display(allFiles)

In [10]:
inspector = inspect(engine)
tables = inspector.get_table_names()
for table in tables:
    print(table)

metadata = MetaData()
metadata.reflect(bind=engine)

def listCols(tablename):
    table = metadata.tables[tablename]
    print(tablename)
    for column in table.columns:
        print(f"   {column}")

listCols('accounts')
listCols('loads')


pd.read_sql(sql=text("select * from loads"), con=engine.connect())   
    